### ConvCRF
[Convolutional CRFs for Semantic Segmentation](https://doi.org/10.48550/arXiv.1805.04777)  
Marvin T. T. Teichmann and Roberto Cipolla

In [1]:
# pip install git+https://github.com/lucasb-eyer/pydensecrf.git

### CryoParticleSegment

In [2]:
#%%bash
#git clone git+https://cyanazuki@github.com/cyanazuki/CryoParticleSegment.git
#cd CryoParticleSegment
#python setup.py install

## ⭐ Setup
You must run all codes under this category.

### ✅ Directory Settings

In [3]:
# @title  { display-mode: "form" }

IMAGE_DIR = "/content/drive/MyDrive/research_xs/processed_micrographs_np" # @param {type:"string"}
LABEL_DIR = "/content/drive/MyDrive/research_xs/ground_truth_mask" # @param {type:"string"}
RESULT_DIR = "/content/drive/MyDrive/research_xs/final/fcn_resnet101/convcrf_with_F" # @param {type:"string"}

In [4]:
# @title  { display-mode: "form" }
# @markdown Detect whether using folder in Google Drive as **`RESULT DIR`**📁.

if "content" in IMAGE_DIR.split("/")[:3] or "content" in LABEL_DIR.split("/")[:3]:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    !rm -r /content/sample_data
    if "content" in IMAGE_DIR.split("/")[:3]:
      !cp -r {IMAGE_DIR} /content/image_dir
      IMAGE_DIR = "/content/image_dir"
    if "content" in LABEL_DIR.split("/")[:3]:
      !cp -r {LABEL_DIR} /content/label_dir
      LABEL_DIR = "/content/label_dir"
  except:
    pass

Mounted at /content/drive


In [5]:
# @title  { display-mode: "form" }
# @markdown Source code directory.
SRC_DIR = "/content/drive/MyDrive/research/src" # @param {type:"string"}

if True:
  !cp -r {SRC_DIR}/EM_project/*.py /content/
else:
  !cp {SRC_DIR}/EM_project/convcrf.py /content/convcrf.py
  !cp {SRC_DIR}/EM_project/dataset.py /content/dataset.py
  !cp {SRC_DIR}/EM_project/lr_scheduler.py /content/lr_scheduler.py
  !cp {SRC_DIR}/EM_project/metrics.py /content/metrics.py
  !cp {SRC_DIR}/EM_project/model.py /content/model.py
  !cp {SRC_DIR}/EM_project/trainer.py /content/trainer.py
  !cp {SRC_DIR}/EM_project/utils.py /content/utils.py
# !rm /content/convcrf.py

### ✅ Packages Handling

In [6]:
# @title  { display-mode: "form" }
# @markdown Useful packages.

import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [7]:
# @title  { display-mode: "form" }
# @markdown User-defined packages.

from dataset import MicrographDataset, MicrographDatasetEvery
from dataset import reconstruct_patched
from model import create_model
from trainer import CryoEMEvaluator
from trainer import CryoEMTrainerWithScheduler, tqdm_plugin_for_Trainer

## ⭐ Main

### ✅ Setting

In [8]:
# @markdown Parameters.

NUM_CLASSES = 2
EPOCHS = 100
BATCH = 7
CROP_SIZE = (512, 512)
LR = 1e-5
RLR_PATIENCE = 3
ES_PATIENCE = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# @markdown Set seed.

random_state = 42
torch.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state)

### ✅ Dataset

In [10]:
train_dir = os.path.join(IMAGE_DIR, 'train')
train_filenames = np.loadtxt(f"{IMAGE_DIR}/train_filenames.txt", dtype=str)
train_dataset = MicrographDataset(image_dir=train_dir, label_dir=LABEL_DIR, filenames=train_filenames, crop_size=CROP_SIZE)

In [11]:
val_dir = os.path.join(IMAGE_DIR, 'val')
val_filenames = np.loadtxt(f"{IMAGE_DIR}/val_filenames.txt", dtype=str)
val_dataset = MicrographDatasetEvery(image_dir=val_dir, label_dir=LABEL_DIR, filenames=val_filenames, crop_size=CROP_SIZE)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False, pin_memory=True)

In [12]:
test_dir = os.path.join(IMAGE_DIR, 'test')
test_filenames = np.loadtxt(f"{IMAGE_DIR}/test_filenames.txt", dtype=str)
test_dataset = MicrographDatasetEvery(image_dir=test_dir, label_dir=LABEL_DIR, filenames=test_filenames, crop_size=CROP_SIZE)
test_loader = DataLoader(test_dataset, batch_size=None, shuffle=False, pin_memory=True)

## ⭐ Convcrf wtih FCN finetuned on cryoem

### ✅ Model

In [13]:
backbone = torch.hub.load('pytorch/vision:v0.10.0', 'fcn_resnet101', num_classes=NUM_CLASSES)

backbone.backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
backbone.classifier[3].p = 0.5

# model = create_model(backbone)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 183MB/s]


In [14]:
CHECKPOINT_PATH = "/content/drive/MyDrive/research_xs/final/fcn_resnet101/checkpoint74_B7_L-3_C512.pt" # @param {type:"string"}
state_dict_path = CHECKPOINT_PATH
state_dict = torch.load(state_dict_path, map_location=torch.device(device))
backbone.load_state_dict(state_dict, strict=False)
backbone.to(device)
backbone.eval()
print("Load model at: ", state_dict_path)

Load model at:  /content/drive/MyDrive/research_xs/final/fcn_resnet101/checkpoint74_B7_L-3_C512.pt


In [15]:
for param in backbone.backbone.parameters():
    param.requires_grad = False

In [16]:
import gc
gc.collect()
torch.cuda.empty_cache()

### ✅ Add convcrf

In [17]:
# @markdown config
config = {
    'filter_size': 7,
    'blur': 4,
    'merge': True,
    'norm': 'none', # sym
    'trainable': False,
    'convcomp': False,

    'weight': 'vector', # scalar
    'unary_weight': 1,
    'weight_init': 0.2,
    'logsoftmax': True,
    'softmax': True,
    'final_softmax': False,

    'pos_feats': {
        'sdims': 3, # 3
        'compat': 3, # 3,
    },

    'col_feats': {
        'sdims': 3, # 80,
        'schan': 0.5, # 13
        'compat': 1, # 10
        'use_bias': False, # True
    },
    "trainable_bias": False,
    "pyinn": False
}

In [18]:
from model import create_crf_model

model = create_crf_model(
    backbone, config=config, shape=CROP_SIZE,
    num_classes=NUM_CLASSES, use_gpu=torch.cuda.is_available()) #crf_args

In [19]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = ReduceLROnPlateau(optimizer, patience=RLR_PATIENCE, verbose=True)

## ⭐ Training

### ✅ Training

In [20]:
Trainer = tqdm_plugin_for_Trainer(CryoEMTrainerWithScheduler)
trainer = Trainer(model, train_dataset, criterion, optimizer, device,
              num_classes = NUM_CLASSES,
              lr_scheduler=scheduler, patience=ES_PATIENCE)

trainer.train(EPOCHS, val_loader=val_loader, batch_size = BATCH,
              ckpt_dir = RESULT_DIR, random_state = random_state)

Epoch   1/100:


Training:   0%|          | 0/9 [00:00<?, ?it/s]

Training score:
  loss	: 1.3277


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

Validation score:
  loss : 1.2254
global correct: 91.06
average row correct: ['96.02', '75.88']
Recall: ['92.41', '86.20']
IoU: ['89.00', '67.66']
F1 Score: ['94.18', '80.71']
mean IoU: 78.33
Saving model at /content/drive/MyDrive/research_xs/final/fcn_resnet101/convcrf_with_F/checkpoint1.pt
Loss improve to 1.225419521331787.
Epoch   2/100:


...

Validation score:
  loss : 1.1379
global correct: 90.81
average row correct: ['96.18', '74.40']
Recall: ['91.99', '86.44']
IoU: ['88.75', '66.62']
F1 Score: ['94.04', '79.97']
mean IoU: 77.69
No improvement for 19 epoch.
Epoch  63/100:


Training:   0%|          | 0/9 [00:00<?, ?it/s]

Training score:
  loss	: 1.1052


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

Validation score:
  loss : 1.1527
global correct: 90.78
average row correct: ['96.27', '74.03']
Recall: ['91.89', '86.64']
IoU: ['88.73', '66.44']
F1 Score: ['94.03', '79.84']
mean IoU: 77.58
Early stopping


In [21]:
import pickle

trainer_result = {
    '_results': trainer._results,
    'best_epoch': trainer.best_epoch,
    'best_loss': trainer.best_loss,
    'loss': trainer.loss,
}

with open(f'{RESULT_DIR}/trainer_result.pickle', 'wb') as f:
    pickle.dump(trainer_result, f)

## ⭐ Evaluation

### ⏭ Test score for all saved checkpoint.

In [ ]:
checkpoint_paths = [path for path in os.listdir(RESULT_DIR) if '.pt' in path]
for checkpoint_path in checkpoint_paths:
  state_dict_path = f"{RESULT_DIR}/{checkpoint_path}"
  state_dict = torch.load(state_dict_path, map_location=torch.device(device))
  model.load_state_dict(state_dict, strict=False)
  model.eval()
  print("\nLoad model at: ", state_dict_path)
  trainer = CryoEMTrainerWithScheduler(model, train_dataset, criterion, optimizer, device,
                num_classes=NUM_CLASSES,
                lr_scheduler=scheduler, patience=ES_PATIENCE)
  result = trainer.evaluate(test_loader)


Load model at:  /content/drive/MyDrive/research_xs/final/fcn_resnet101/fcn_resnet101_B7_LR-5_CS512/Crf_model_convcrf_with_F/checkpoint1.pt
  loss : 31.6959
  Average Precision : 0.1185
global correct: 86.98
average row correct: ['95.31', '58.08']
Recall: ['88.75', '78.13']
IoU: ['85.04', '49.96']
F1 Score: ['91.91', '66.63']
mean IoU: 67.50

Load model at:  /content/drive/MyDrive/research_xs/final/fcn_resnet101/fcn_resnet101_B7_LR-5_CS512/Crf_model_convcrf_with_F/checkpoint2.pt
  loss : 31.4424
  Average Precision : 0.1204
global correct: 87.09
average row correct: ['95.18', '59.03']
Recall: ['88.96', '77.91']
IoU: ['85.12', '50.57']
F1 Score: ['91.96', '67.17']
mean IoU: 67.85

Load model at:  /content/drive/MyDrive/research_xs/final/fcn_resnet101/fcn_resnet101_B7_LR-5_CS512/Crf_model_convcrf_with_F/checkpoint4.pt
  loss : 31.3555
  Average Precision : 0.1209
global correct: 87.11
average row correct: ['95.14', '59.28']
Recall: ['89.02', '77.85']
IoU: ['85.14', '50.72']
F1 Score: ['9

### ⏭ Load previous

In [22]:
# @markdown Set show_checkpoint to True to check every checkpoint.
# @markdown If checkpoint_filename is "" (empty), the last checkpoint will be used.
show_checkpoint = False # @param {type:"boolean"}
checkpoint_filename = "" # @param {type:"string"}
if True:
  checkpoint_paths = [path for path in os.listdir(RESULT_DIR) if '.pt' in path]
  if show_checkpoint:
    print(checkpoint_paths)
  if str(checkpoint_filename) == "":
    checkpoint_filename = checkpoint_paths[-1]
  state_dict_path = f"{RESULT_DIR}/{checkpoint_filename}"
  state_dict = torch.load(state_dict_path, map_location=torch.device(device))
  model.load_state_dict(state_dict, strict=False)
  model.eval()
  print("Load model at: ", state_dict_path)
  try:
    # Run if changing trainer.
    import pickle
    # !rm __pycache__/*
    # !cp /content/drive/MyDrive/research/src/EM_project/trainer.py /content/trainer.py
    # from trainer import CryoEMTrainerWithScheduler
    Trainer = tqdm_plugin_for_Trainer(CryoEMTrainerWithScheduler)
    trainer = Trainer(model, train_dataset, val_dataset, criterion, optimizer, device,
                  num_classes = NUM_CLASSES,
                  lr_scheduler=scheduler, patience=ES_PATIENCE)
    with open(f'{RESULT_DIR}/trainer_result.pickle', 'rb') as f:
      trainer_result = pickle.load(f)
    for key in trainer_result:
      setattr(trainer, key, trainer_result[key])
    # for var in dir(trainer):
    #   if '__' not in var:
    #     print(var)
    #     print(f'var:\n  ', getattr(trainer, var))
  except:
    pass

Load model at:  /content/drive/MyDrive/research_xs/final/fcn_resnet101/convcrf_with_F/checkpoint43.pt


### ✅ Testing

In [23]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
evaluator = CryoEMEvaluator(model=model, device=device, metrics=['iou'], num_classes=NUM_CLASSES)
print("FCN validation result:")
result = evaluator.evaluate(loader=val_loader)
print("FCN test result:")
result = evaluator.evaluate(loader=test_loader)

FCN validation result:
global correct: 90.97
average row correct: ['95.85', '76.04']
Recall: ['92.44', '85.71']
IoU: ['88.89', '67.48']
F1 Score: ['94.12', '80.59']
mean IoU: 78.19
FCN test result:
global correct: 89.99
average row correct: ['95.35', '71.38']
Recall: ['92.04', '81.56']
IoU: ['88.08', '61.46']
F1 Score: ['93.66', '76.13']
mean IoU: 74.77


In [25]:
from torchvision.utils import save_image
from dataset import reconstruct_patched

!mkdir {RESULT_DIR}/test_image
model.eval()
with torch.no_grad():
  for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    inputs = test_image.to(device)
    outputs = model(inputs)['out']
    preds = outputs.argmax(dim=1).cpu().detach()
    filename = f"{os.path.splitext(test_dataset.filenames[idx])[0]}.png"
    pred_path = os.path.join(RESULT_DIR, "test_image", filename)
    save_image(reconstruct_patched(preds, grid).float(), pred_path)